Setting up

In [ ]:
import sklearn
import pandas as pd
import numpy as np

input_prefix = "../input/anz_crash_20200903_fix/"
cas_fp = input_prefix + "Casualties.csv"
crash_fp = input_prefix + "Crash2.csv"
dt_fp = input_prefix + "DateTime.csv"
desc_fp = input_prefix + "Description.csv"
loc_fp = input_prefix + "Location2.csv"
veh_fp = input_prefix + "Vehicles.csv"

Crash.csv is the "central" csv file that the other csv files connect to.

Rows in crash.csv contain id values that link to description, datetime, location, casualties, and vehicles.

The dataset source is https://www.kaggle.com/datasets/mgray39/australia-new-zealand-road-crash-dataset?resource=download-directory&select=anz_crash_20200903_fix and the link contains a graphic which clearly explains how all the files link together.

In order to make this data usable, all the data needs to be combined using the id information.

Combine all csv files together

In [ ]:
cas_dat = pd.read_csv(cas_fp)
crash_dat = pd.read_csv(crash_fp)
dt_dat = pd.read_csv(dt_fp)
desc_dat = pd.read_csv(desc_fp)
loc_dat = pd.read_csv(loc_fp)
veh_dat = pd.read_csv(veh_fp)